In [1]:
import pandas as pd

In [2]:
df_fe_grids = pd.read_csv(
    'Data/Processed_Datasets/Combined_tables/df_grids_keys.csv')
df_crimes = pd.read_csv(
    'Data/Processed_Datasets/Cleaned_datasets/df_crimes.csv')

In [3]:
df_crimes.columns

Index(['Unnamed: 0', 'CATEGORIE', 'DATE', 'QUART', 'LONGITUDE', 'LATITUDE',
       'DAY', 'MONTH', 'YEAR', '1000_GRID_ID', 'CRIMES_ID'],
      dtype='object')

In [4]:
df_fe_grids.sort_values(['1000_GRID_ID', 'MONTH'])

,Unnamed: 0,1000_GRID_ID,MONTH,CODEMAMROT
0,0,-1xcNei0iJs=,2015-01,REM34
1,1,-1xcNei0iJs=,2015-02,REM34
2,2,-1xcNei0iJs=,2015-03,REM34
3,3,-1xcNei0iJs=,2015-04,REM34
4,4,-1xcNei0iJs=,2015-05,REM34
...,...,...,...,...
69253,69253,zzht2gM06A0=,2022-09,REM17
69254,69254,zzht2gM06A0=,2022-10,REM17
69255,69255,zzht2gM06A0=,2022-11,REM17
69256,69256,zzht2gM06A0=,2022-12,REM17


In [5]:
df_fe_grids = df_fe_grids[['1000_GRID_ID', 'MONTH', 'CODEMAMROT']]

### Number of crime records per grid and month. 

In [6]:
crime_counts = df_crimes.groupby(
    ['1000_GRID_ID', 'MONTH']).size().reset_index(name='CRIME_COUNT')

In [7]:
df_fe_crimes = df_fe_grids.merge(crime_counts,
                                        on=['1000_GRID_ID', 'MONTH'],
                                        how='left')

In [8]:
df_fe_crimes['CRIME_COUNT'] = df_fe_crimes['CRIME_COUNT'].fillna(0)

In [9]:
df_fe_crimes.describe(include='all')

,1000_GRID_ID,MONTH,CODEMAMROT,CRIME_COUNT
count,69258,69258,69258,69258.000000
unique,714,97,34,NaN
top,ih4zU-dNtzo=,2021-09,REM33,NaN
freq,97,714,5723,NaN
mean,NaN,NaN,NaN,2.929019
std,NaN,NaN,NaN,5.416797
min,NaN,NaN,NaN,0.000000
25%,NaN,NaN,NaN,0.000000
50%,NaN,NaN,NaN,1.000000
75%,NaN,NaN,NaN,4.000000


### Number of incidents per grid and month for each category (6)

In [10]:
crime_counts = pd.pivot_table(df_crimes, index=[
    '1000_GRID_ID', 'MONTH'], columns='CATEGORIE', values='Unnamed: 0', aggfunc='count')

In [11]:
crime_counts.columns = [
    f'{col}_CRIME_COUNT' for col in crime_counts.columns]
df_fe_crimes = df_fe_crimes.join(crime_counts, on=['1000_GRID_ID', 'MONTH'])
df_fe_crimes = df_fe_crimes.fillna(0)

In [12]:
df_fe_crimes.describe(include='all')

,1000_GRID_ID,MONTH,CODEMAMROT,CRIME_COUNT,Infractions entrainant la mort_CRIME_COUNT,Introduction_CRIME_COUNT,Méfait_CRIME_COUNT,Vol dans / sur véhicule à moteur_CRIME_COUNT,Vol de véhicule à moteur_CRIME_COUNT,Vols qualifiés_CRIME_COUNT
count,69258,69258,69258,69258.000000,69258.000000,69258.000000,69258.000000,69258.000000,69258.000000,69258.000000
unique,714,97,34,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,ih4zU-dNtzo=,2021-09,REM33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,97,714,5723,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,2.929019,0.002671,0.842589,0.640114,0.773802,0.514525,0.155318
std,NaN,NaN,NaN,5.416797,0.052995,1.990643,1.531543,1.907677,1.150850,0.557069
min,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,NaN,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,NaN,NaN,NaN,4.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000


In [13]:
sums = df_fe_crimes.select_dtypes(exclude=['object']).sum()
sums

CRIME_COUNT                                     202858.0
Infractions entrainant la mort_CRIME_COUNT         185.0
Introduction_CRIME_COUNT                         58356.0
Méfait_CRIME_COUNT                               44333.0
Vol dans / sur véhicule à moteur_CRIME_COUNT     53592.0
Vol de véhicule à moteur_CRIME_COUNT             35635.0
Vols qualifiés_CRIME_COUNT                       10757.0
dtype: float64

# Output file

In [14]:
df_fe_crimes.to_csv(
    'Data/Processed_Datasets/Combined_tables/df_fe_crimes.csv')